In [2]:
# Импортируем библиотеки

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error

In [0]:
def split_sequence(sequence, n_steps):
    n = len(sequence)
    X, y = list(), list()
    for i in range(n):
        end_ix = i + n_steps
        if end_ix > n-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [0]:
def plot_difference(true_sequence, predict_values, unit):
    print(f'Prediction of model with {unit}:')
    df = pd.DataFrame({'real value': true_sequence, 'predicted value': np.round(predict_values, 3), 'difference': np.round(abs(true_sequence-predict_values), 1)})
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(df)

# **Data generation**

In [0]:
def fibonacci(start=(0, 1), num=10):
    x = [start[0], start[1]]
    for i in range(2, num+1):
        x.append(x[i-1] + x[i-2])
    return np.array(x)

In [0]:
def generate_data(n_steps=4):
    
    num = 25
    batch_size = 2000
    i = 0
    
    X = np.empty((batch_size, n_steps))
    y = np.empty((batch_size, 1))

    start_val = np.array([0, 1])

    while i < batch_size:
    
        array = fibonacci(start=(start_val[0], start_val[1]), num=num)
        j = 0

        while j <= num - (n_steps + 1):
            if i >= batch_size: break
            X[i, :] = array[j:n_steps+j]
            y[i, :] = array[j+n_steps] 
            i += 1
            j += 1
        
        start_val += 1

    _, index = np.unique(X, axis=0, return_index=True)
    X, y = X[index], y[index]
    X = X.reshape((X.shape[0], X.shape[1], 1))
    return X, y

In [0]:
n_steps = 4
X_train, y_train = generate_data(n_steps)
print(f'X_train shape is {X_train.shape}')
print(f'y_train shape is {y_train.shape}')

X_train shape is (1961, 4, 1)
y_train shape is (1961, 1)


## **Model with one LSTM layer**

In [0]:
def predict(true_sequence, model):
    predict_values = true_sequence[:n_steps]
    k = n_steps
    length = true_sequence.size
    while k != length:
        X = predict_values[-n_steps::]
        X = X.reshape((1, n_steps, 1))
        f_x = model.predict(X, verbose=0) 
        predict_values = np.append(predict_values, f_x)
        k += 1
    return predict_values

In [0]:
sequence = fibonacci(start=(2, 2), num=25)

In [0]:
units_number = list(range(1, 11)) + list(range(15, 65, 5))
losses = []
min_loss = None
min_unit = None

for unit in units_number: 
    model = Sequential()
    model.add(LSTM(unit, activation='softplus', input_shape=(n_steps, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    model.fit(X_train, y_train, epochs=800, validation_split=0.2, verbose=0)
    
    predict_y = predict(sequence, model)
    mse = mean_squared_error(sequence, predict_y)

    if min_loss is None or mse < min_loss:
        min_unit = unit
        min_loss = mse
        best_model = model
    losses.append(mse)
    
    print(f'Units in LSTM layer: {unit}, MSE is:: {np.round(mse, 4)}')
    if unit in (5, 6, 7, 8, 9, 40, 50):
        plot_difference(sequence, predict_y, unit)

Units in LSTM layer: 1, MSE is:: 2399434630.5131
Units in LSTM layer: 2, MSE is:: 100525294.538
Units in LSTM layer: 3, MSE is:: 18464.1777
Units in LSTM layer: 4, MSE is:: 44394.6597
Units in LSTM layer: 5, MSE is:: 128990.2773
Prediction of model with 5:
    real value  predicted value  difference
0            1            1.000         0.0
1            2            2.000         0.0
2            3            3.000         0.0
3            5            5.000         0.0
4            8            8.019         0.0
5           13           13.041         0.0
6           21           20.691         0.3
7           34           34.215         0.2
8           55           54.477         0.5
9           89           87.595         1.4
10         144          142.683         1.3
11         233          231.493         1.5
12         377          372.490         4.5
13         610          605.548         4.5
14         987          982.136         4.9
15        1597         1585.507        

# **Testing best model performance**

$F_n = F_{n-1} + F_{n-2}, n >= 2$  
Пусть $F_0 = 0, F_1 = 1$

In [0]:
sequence = fibonacci(start=(0, 1), num=25)
predict_y = predict(sequence, best_model)
mse = mean_squared_error(sequence, predict_y)
print(f'mse is {mse}')
plot_difference(sequence, predict_y, min_unit)

mse is 31027.72803814561
Prediction of model with 8:
    real value  predicted value  difference
0            0            0.000         0.0
1            1            1.000         0.0
2            1            1.000         0.0
3            2            2.000         0.0
4            3            2.969         0.0
5            5            4.923         0.1
6            8            7.915         0.1
7           13           12.861         0.1
8           21           20.771         0.2
9           34           33.691         0.3
10          55           54.496         0.5
11          89           88.211         0.8
12         144          142.651         1.3
13         233          230.865         2.1
14         377          373.346         3.7
15         610          604.231         5.8
16         987          977.414         9.6
17        1597         1581.913        15.1
18        2584         2559.576        24.4
19        4181         4141.449        39.6
20        6765         

Пусть $F_0 = 4, F_1=5$

In [0]:
sequence = fibonacci(start=(4, 5), num=25)
predict_y = predict(sequence, best_model)
mse = mean_squared_error(sequence, predict_y)
print(f'mse is {mse}')
plot_difference(sequence, predict_y, min_unit)

mse is 147027.77107351896
Prediction of model with 8:
    real value  predicted value  difference
0            4            4.000         0.0
1            5            5.000         0.0
2            9            9.000         0.0
3           14           14.000         0.0
4           23           23.005         0.0
5           37           37.073         0.1
6           60           60.147         0.1
7           97           97.210         0.2
8          157          157.410         0.4
9          254          254.758         0.8
10         411          411.882         0.9
11         665          666.519         1.5
12        1076         1079.030         3.0
13        1741         1745.568         4.6
14        2817         2824.479         7.5
15        4558         4570.405        12.4
16        7375         7395.021        20.0
17       11933        11965.442        32.4
18       19308        19360.699        52.7
19       31241        31326.346        85.3
20       50549        

# **Model with two LSTM layers**

In [0]:
n_steps = 4
X_train, y_train = generate_data(n_steps)
print(f'X_train shape is {X_train.shape}')
print(f'y_train shape is {y_train.shape}')

X_train shape is (1961, 4, 1)
y_train shape is (1961, 1)


In [0]:
units_number = list(range(1, 11)) + list(range(15, 65, 5))
losses = []
min_loss = None
min_unit = None

for unit in units_number: 
    model = Sequential()
    model.add(LSTM(unit, activation='softplus', return_sequences=True, input_shape=(n_steps, 1)))
    model.add(LSTM(unit, activation='softplus'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    model.fit(X_train, y_train, epochs=800, validation_split=0.2, verbose=0)
    
    predict_y = predict(sequence, model)
    mse = mean_squared_error(sequence, predict_y)

    if min_loss is None or mse < min_loss:
        min_unit = unit
        min_loss = mse
        best_model = model
    losses.append(mse)
    
    print(f'Units in LSTM layer: {unit}, MSE is:: {np.round(mse, 4)}')
    if unit in (5, 6, 7, 8, 9, 40, 50):
        plot_difference(sequence, predict_y, unit)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Units in LSTM layer: 1, MSE is:: 543384195.084
Units in LSTM layer: 2, MSE is:: 3666441733.9008
Units in LSTM layer: 3, MSE is:: 20419857.0409
Units in LSTM layer: 4, MSE is:: 2752561.5383
Units in LSTM layer: 5, MSE is:: 35766405974.386
Prediction of model with 5:
    real value  predicted value  difference
0            2            2.000         0.0
1            2            2.000         0.0
2            4            4.000         0.0
3            6            6.000         0.0
4           10           10.980         1.0
5           16           16.469         0.5
6           26           26.989         1.0
7           42           44.543         2.5
8           68           72.580         4.6
9          110          117.571         7.6
10         178          191.963        14.0
11         288          321.694

# **Testing best model performance**

In [0]:
sequence = fibonacci(start=(0, 1), num=25)
predict_y = predict(sequence, best_model)
mse = mean_squared_error(sequence, predict_y)
print(f'mse is {mse}')
plot_difference(sequence, predict_y, min_unit)

mse is 74231.00849568703
Prediction of model with 50:
    real value  predicted value  difference
0            0            0.000         0.0
1            1            1.000         0.0
2            1            1.000         0.0
3            2            2.000         0.0
4            3            2.945         0.1
5            5            4.928         0.1
6            8            7.876         0.1
7           13           12.806         0.2
8           21           20.693         0.3
9           34           33.504         0.5
10          55           54.237         0.8
11          89           87.791         1.2
12         144          141.876         2.1
13         233          229.632         3.4
14         377          371.694         5.3
15         610          601.046         9.0
16         987          972.434        14.6
17        1597         1574.029        23.0
18        2584         2546.520        37.5
19        4181         4119.783        61.2
20        6765        

In [0]:
sequence = fibonacci(start=(4, 5), num=25)
predict_y = predict(sequence, best_model)
mse = mean_squared_error(sequence, predict_y)
print(f'mse is {mse}')
plot_difference(sequence, predict_y, min_unit)

mse is 3093.770385513009
Prediction of model with 50:
    real value  predicted value  difference
0            4            4.000         0.0
1            5            5.000         0.0
2            9            9.000         0.0
3           14           14.000         0.0
4           23           23.007         0.0
5           37           37.017         0.0
6           60           60.034         0.0
7           97           97.038         0.0
8          157          156.852         0.1
9          254          253.927         0.1
10         411          410.853         0.1
11         665          664.877         0.1
12        1076         1075.351         0.6
13        1741         1740.231         0.8
14        2817         2815.936         1.1
15        4558         4555.651         2.3
16        7375         7371.206         3.8
17       11933        11927.281         5.7
18       19308        19298.604         9.4
19       31241        31225.939        15.1
20       50549        

In [0]:
sequence = fibonacci(start=(3, 3), num=25)
predict_y = predict(sequence, best_model)
mse = mean_squared_error(sequence, predict_y)
print(f'mse is {mse}')
plot_difference(sequence, predict_y, min_unit)

mse is 2605.724642146611
Prediction of model with 50:
    real value  predicted value  difference
0            3            3.000         0.0
1            3            3.000         0.0
2            6            6.000         0.0
3            9            9.000         0.0
4           15           14.972         0.0
5           24           23.986         0.0
6           39           39.057         0.1
7           63           62.993         0.0
8          102          101.936         0.1
9          165          164.773         0.2
10         267          267.051         0.1
11         432          431.724         0.3
12         699          698.600         0.4
13        1131         1130.325         0.7
14        1830         1828.954         1.0
15        2961         2959.244         1.8
16        4791         4787.868         3.1
17        7752         7746.966         5.0
18       12543        12534.983         8.0
19       20295        20282.064        12.9
20       32838        